In [1]:
%load_ext autoreload
# import datetime
# import ete3
# import itertools
import json
import logging
import math
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
# import os
import pandas as pd
import typing
# import re
import xml.etree.ElementTree as ET
# import xml.dom.minidom
# import xmlschema
import pm4py

In [2]:
%autoreload
from maxes.xes_loader2 import XesLoader
# from maxes.xes_file import XesFile
# from maxes.analyze_xes import AnalyzeXes
# import maxes.analyze_sequence
# import maxes.serialization.serialize
# import maxes.graphs
import maxes.notebooks.utils
# import maxes.utils
# from maxes.analyze_xes import AnalyzeXes

In [ ]:
maxes.notebooks.utils.init_notebook() # RUN ONLY ONCE

In [4]:
%autoreload
from maxes.xes_loader2 import XesLoader
import maxes.notebooks.utils

file_path = maxes.notebooks.utils.get_data_path('other/event_log.xes')

log = XesLoader().load(file_path)
log.df

,case:concept:name,concept:name,lifecycle:transition,time:timestamp,Payment type,Products,instance,Order ID,action,model,type,Customer type,Number of orders,Shipment ID,Courier,Amount,Replenishment ID,Product
0,1,,complete,2023-12-31 23:02:57+00:00,Credit,"[laptop,phone,laptop,desktop]",1,1.0,Receive customer order,Order,Order,Silver,NaN,NaN,NaN,NaN,NaN,NaN
1,1,,complete,2023-12-31 23:14:11+00:00,Plan,"[laptop,desktop]",2,2.0,Receive customer order,Order,Order,Bronze,NaN,NaN,NaN,NaN,NaN,NaN
2,1,,complete,2023-12-31 23:14:11+00:00,Credit,"[laptop,phone,laptop,desktop]",1,1.0,Try payment,Order,Order,Silver,NaN,NaN,NaN,NaN,NaN,NaN
3,1,,complete,2023-12-31 23:16:01+00:00,Plan,"[laptop,desktop]",2,2.0,Deny order,Order,Order,Bronze,NaN,NaN,NaN,NaN,NaN,NaN
4,1,,complete,2023-12-31 23:16:01+00:00,Credit,[phone],3,3.0,Receive customer order,Order,Order,Gold,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31250,1,,complete,2024-01-31 22:50:39+00:00,Plan,"[tv,desktop]",2970,2970.0,Complete packing,Order,Order,Bronze,NaN,NaN,NaN,NaN,NaN,NaN
31251,1,,complete,2024-01-31 22:56:07+00:00,Plan,"[phone,tv,laptop,phone]",2998,2998.0,Pick inventory,Order,Order,Bronze,NaN,NaN,NaN,NaN,NaN,NaN
31252,1,,complete,2024-01-31 22:56:07+00:00,Plan,"[phone,tv,laptop,phone]",2998,2998.0,Pick inventory,Order,Order,Bronze,NaN,NaN,NaN,NaN,NaN,NaN
31253,1,,complete,2024-01-31 22:56:07+00:00,Plan,"[phone,tv,laptop,phone]",2998,2998.0,Pick inventory,Order,Order,Bronze,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
%autoreload
from maxes.utils import reorder_xes_columns

log_df_processed = (
    log.df
    .copy()
    .drop(columns=['case:concept:name', 'concept:name'])
    .rename(columns={'instance': 'case:concept:name', 'action': 'concept:name'})
    .sort_values(by=['case:concept:name', 'time:timestamp'])
    .reset_index(drop=True)
)

# reorder columns
log_df_processed = reorder_xes_columns(log_df_processed)

log_df_processed

,case:concept:name,concept:name,lifecycle:transition,time:timestamp,Payment type,Products,Order ID,model,type,Customer type,Number of orders,Shipment ID,Courier,Amount,Replenishment ID,Product
0,1,Receive customer order,complete,2023-12-31 23:02:57+00:00,Credit,"[laptop,phone,laptop,desktop]",1.0,Order,Order,Silver,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Try payment,complete,2023-12-31 23:14:11+00:00,Credit,"[laptop,phone,laptop,desktop]",1.0,Order,Order,Silver,NaN,NaN,NaN,NaN,NaN,NaN
2,1,Receive payment,complete,2023-12-31 23:19:10+00:00,Credit,"[laptop,phone,laptop,desktop]",1.0,Order,Order,Silver,NaN,NaN,NaN,NaN,NaN,NaN
3,1,Process order,complete,2023-12-31 23:24:14+00:00,Credit,"[laptop,phone,laptop,desktop]",1.0,Order,Order,Silver,NaN,NaN,NaN,NaN,NaN,NaN
4,1,Pick inventory,complete,2024-01-01 00:04:11+00:00,Credit,"[laptop,phone,laptop,desktop]",1.0,Order,Order,Silver,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31250,2998,Process order,complete,2024-01-31 22:16:24+00:00,Plan,"[phone,tv,laptop,phone]",2998.0,Order,Order,Bronze,NaN,NaN,NaN,NaN,NaN,NaN
31251,2998,Pick inventory,complete,2024-01-31 22:56:07+00:00,Plan,"[phone,tv,laptop,phone]",2998.0,Order,Order,Bronze,NaN,NaN,NaN,NaN,NaN,NaN
31252,2998,Pick inventory,complete,2024-01-31 22:56:07+00:00,Plan,"[phone,tv,laptop,phone]",2998.0,Order,Order,Bronze,NaN,NaN,NaN,NaN,NaN,NaN
31253,2998,Pick inventory,complete,2024-01-31 22:56:07+00:00,Plan,"[phone,tv,laptop,phone]",2998.0,Order,Order,Bronze,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
%autoreload
from maxes.log_builder import LogBuilder

log_builder = LogBuilder()
log_processed = log_builder.build_log(log_df_processed)
log_processed

In [7]:
log_processed.update_df()
log_processed.df

,case:concept:name,concept:name,lifecycle:transition,time:timestamp,Payment type,Products,Order ID,model,type,Customer type,Number of orders,Shipment ID,Courier,Amount,Replenishment ID,Product
0,1,Receive customer order,complete,2023-12-31 23:02:57+00:00,Credit,"[laptop,phone,laptop,desktop]",1.0,Order,Order,Silver,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Try payment,complete,2023-12-31 23:14:11+00:00,Credit,"[laptop,phone,laptop,desktop]",1.0,Order,Order,Silver,NaN,NaN,NaN,NaN,NaN,NaN
2,1,Receive payment,complete,2023-12-31 23:19:10+00:00,Credit,"[laptop,phone,laptop,desktop]",1.0,Order,Order,Silver,NaN,NaN,NaN,NaN,NaN,NaN
3,1,Process order,complete,2023-12-31 23:24:14+00:00,Credit,"[laptop,phone,laptop,desktop]",1.0,Order,Order,Silver,NaN,NaN,NaN,NaN,NaN,NaN
4,1,Pick inventory,complete,2024-01-01 00:04:11+00:00,Credit,"[laptop,phone,laptop,desktop]",1.0,Order,Order,Silver,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,2998,Process order,complete,2024-01-31 22:16:24+00:00,Plan,"[phone,tv,laptop,phone]",2998.0,Order,Order,Bronze,NaN,NaN,NaN,NaN,NaN,NaN
4,2998,Pick inventory,complete,2024-01-31 22:56:07+00:00,Plan,"[phone,tv,laptop,phone]",2998.0,Order,Order,Bronze,NaN,NaN,NaN,NaN,NaN,NaN
5,2998,Pick inventory,complete,2024-01-31 22:56:07+00:00,Plan,"[phone,tv,laptop,phone]",2998.0,Order,Order,Bronze,NaN,NaN,NaN,NaN,NaN,NaN
6,2998,Pick inventory,complete,2024-01-31 22:56:07+00:00,Plan,"[phone,tv,laptop,phone]",2998.0,Order,Order,Bronze,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
%autoreload
from maxes.generators.xes_generator.xes_generator3 import XesGenerator3 as XesGenerator

generator = XesGenerator(generate_traces_count=1, debug=True)
generator.fit(log_processed)

In [9]:
generator.generate_traces_count = 10
result_log = generator.sample(1)[0]
reorder_xes_columns(result_log.df)
# result_log.df

,case:concept:name,concept:name,lifecycle:transition,time:timestamp,Products,Customer type,Payment type,model,type,Number of orders,Shipment ID,Courier,Amount,Replenishment ID,Product
0,0,Receive customer order,complete,2024-01-03 04:53:58.290869+00:00,"[phone,tv,laptop]",Gold,Credit,Order,Order,NaN,NaN,NaN,NaN,NaN,NaN
1,0,Try payment,complete,2024-01-03 05:02:45.089504+00:00,"[phone,tv,laptop]",Gold,Debit,Order,Order,NaN,NaN,NaN,NaN,NaN,NaN
2,0,Receive payment,complete,2024-01-03 05:07:39.624446+00:00,"[phone,tv,laptop]",Gold,Credit,Order,Order,NaN,NaN,NaN,NaN,NaN,NaN
3,0,Process order,complete,2024-01-03 05:12:51.297088+00:00,"[phone,tv,laptop]",Gold,Credit,Order,Order,NaN,NaN,NaN,NaN,NaN,NaN
4,0,Pick inventory,complete,2024-01-03 05:22:51.602810+00:00,"[phone,tv,laptop]",Gold,Credit,Order,Order,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,9,Pick inventory,complete,2024-01-29 13:51:37.405389+00:00,"[tv,desktop,phone]",Silver,Credit,Order,Order,NaN,NaN,NaN,NaN,NaN,NaN
85,9,Pick backorder,complete,2024-01-29 19:35:08.350924+00:00,"[tv,desktop,phone]",Silver,Credit,Order,Order,NaN,NaN,NaN,NaN,NaN,NaN
86,9,Pack order,complete,2024-01-29 20:01:50.519616+00:00,"[tv,desktop,phone]",Silver,Debit,Order,Order,NaN,NaN,NaN,NaN,NaN,NaN
87,9,Complete packing,complete,2024-01-29 20:16:54.083816+00:00,"[tv,desktop,phone]",Silver,Debit,Order,Order,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# TODO: Generate Order ID

del result_log.event_attributes_info['Order ID']

result_log.event_attributes_info

{'concept:name': <maxes.xes_log.AttributeInfo at 0x35808eb70>,
 'time:timestamp': <maxes.xes_log.AttributeInfo at 0x35808ee10>,
 'lifecycle:transition': <maxes.xes_log.AttributeInfo at 0x358075430>,
 'Payment type': <maxes.xes_log.AttributeInfo at 0x358075cd0>,
 'Products': <maxes.xes_log.AttributeInfo at 0x35808ffb0>,
 'model': <maxes.xes_log.AttributeInfo at 0x358090290>,
 'type': <maxes.xes_log.AttributeInfo at 0x3580907d0>,
 'Customer type': <maxes.xes_log.AttributeInfo at 0x3580913d0>,
 'Number of orders': <maxes.xes_log.AttributeInfo at 0x358090830>,
 'Shipment ID': <maxes.xes_log.AttributeInfo at 0x358091430>,
 'Courier': <maxes.xes_log.AttributeInfo at 0x358091490>,
 'Amount': <maxes.xes_log.AttributeInfo at 0x3580914f0>,
 'Replenishment ID': <maxes.xes_log.AttributeInfo at 0x358091550>,
 'Product': <maxes.xes_log.AttributeInfo at 0x3580915b0>}

In [11]:
result_log.update_traces_from_df()

In [12]:
# Serialize

%autoreload

import os

from maxes.serialization.serialize import Serializer

generated_log_ET = Serializer().serialize(result_log)

ET.indent(generated_log_ET)
ET.register_namespace("", "http://www.xes-standard.org")

generated_log_path = os.path.join(maxes.notebooks.utils.get_project_path(), "output/event_log.xes")
with open(generated_log_path, "w") as file:
    generated_log_ET.write(file, encoding="unicode")

result_log